In [ ]:
# pip install --upgrade torch

In [ ]:
# pip install dgl.sparse 

In [ ]:
# pip install dgl==0.9

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import scipy.sparse as sp

import dgl
import numpy as np
import torch
import torch.nn.functional as F
from dgl.dataloading import GraphDataLoader
from sklearn.model_selection import train_test_split
from time import time
from tqdm import tqdm
from sklearn.manifold import TSNE
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA

from functions_HGPLS import train, test

In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("seyonec/SMILES_tokenized_PubChem_shard00_160k")

model = AutoModelForMaskedLM.from_pretrained("seyonec/SMILES_tokenized_PubChem_shard00_160k")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
atoms = ["", "H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne", "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm"]
print(len(atoms))
ids = tokenizer(atoms).input_ids

toks = list()
for i in ids:
    if len(i) >= 3:
        toks.append(i[1])
    else:
        toks.append(i[0])

100


In [4]:
dataset, labels = dgl.load_graphs("../data/HIV_dgl_graphs")

In [5]:
labels = labels['glabel'].tolist()

In [6]:
# d = dataset[0].ndata["atomic"]
tok = torch.zeros((len(toks), 1))
for i in range(len(toks)):
    tok[i] = toks[i]

tok
transfo_atoms = model(tok.to(torch.long)).logits.reshape(100, -1)

In [7]:
pca = PCA(n_components=100, random_state=42)
res = pca.fit_transform(transfo_atoms.detach().numpy())

In [8]:
new_dataset = list()
for i in range(len(dataset)):
    #g, l = dataset[i]
    g = dataset[i]
    l = labels[i]
    #g.ndata["feature"] = torch.ones_like(dataset.feature[g.ndata["_ID"]])
    ids=list(g.ndata["atomic"].numpy().flatten().astype(int))
    g.ndata["feature"] = torch.tensor(res[ids]) #g.ndata["atomic"]
    g = dgl.add_self_loop(g)
    g = dgl.add_reverse_edges(g)
    new_dataset.append((g, l))

In [9]:
train_dataset, test_dataset = train_test_split(new_dataset, test_size=0.25, random_state=42)

train_dataloader = GraphDataLoader(train_dataset, batch_size=16, drop_last=False)
test_dataloader = GraphDataLoader(test_dataset, batch_size=16, drop_last=False)

In [10]:
from dgl.nn.pytorch import GATConv

class GAT(nn.Module):
    def __init__(self, in_feats, h_feats, n_classes):
        super(GAT, self).__init__()
        self.layer1 = GATConv(in_feats, h_feats, num_heads=4)
        self.layer2 = GATConv(4*h_feats, h_feats, num_heads=4)
        self.layer3 = GATConv(4*h_feats, h_feats, num_heads=6)
        self.fc = nn.Linear(h_feats, n_classes)
        self.elu = nn.ELU()

    def forward(self, g, in_feat):
        x1 = self.layer1(g, in_feat)
        x1 = self.elu(x1)
        x1 = x1.view(in_feat.shape[0], -1)
        x2 = self.layer2(g, x1)
        x2 = self.elu(x2)
        x2 = x2.view(in_feat.shape[0], -1)
        x3 = self.layer3(g, x2)
        x3 = torch.mean(x3, dim=1)
        with g.local_scope():
            g.ndata['h'] = x3
            x4 = dgl.readout_nodes(g, 'h')
        return F.log_softmax(self.fc(x4), dim=-1)

In [11]:
# Load model architecture
device = 'cpu' if torch.cuda.is_available() else 'cpu'
model = GAT(in_feats=100, n_classes=3, h_feats=256).to(device)

In [ ]:
model.load_state_dict(torch.load("../models/GATModel_prot.pt"))

In [12]:
# Define optimizer and loss
optimizer = torch.optim.Adam(
        model.parameters(), lr=0.000002 #, weight_decay=0.0001
    )
loss = torch.nn.CrossEntropyLoss()

In [13]:
# Train model and keep the best validation loss model
bad_cound = 0
best_val_acc = 0
best_epoch = 0
epochs = 40
patience = 10
print_every = 1
train_times = []
for e in range(epochs):
    s_time = time()
    train_loss, train_acc = train(model, optimizer, loss, train_dataloader, device)
    train_times.append(time() - s_time)
    val_acc, val_loss = test(model, loss, test_dataloader, device)
    if best_val_acc < val_acc:
        best_val_acc = val_acc
        bad_cound = 0
        best_epoch = e + 1
        torch.save(model.state_dict(), "../models/GATModel_prot.pt")
    else:
        bad_cound += 1
    if bad_cound >= patience:
        break

    if (e + 1) % print_every == 0:
        log_format = (
            "Epoch {}: train_loss={:.4f}, train_acc={:.4f}, val_acc={:.4f}, vall_loss={:.4f}"
        )
        print(log_format.format(e + 1, train_loss, train_acc, val_acc, val_loss))
print(
    "Best Epoch {}, final test loss {:.4f}".format(
        best_epoch, best_val_acc
    )
)

100%|██████████| 39/39 [00:02<00:00, 15.57it/s]


Epoch 1: train_loss=30.1189, train_acc=0.3564, val_acc=0.3961, vall_loss=6.8792


 34%|███▍      | 39/115 [00:07<00:14,  5.10it/s]

In [51]:
pred=[]
lab = []
for batch in tqdm(test_dataloader):
        batch_graphs, batch_labels = batch
        batch_graphs = batch_graphs.to(device)
        batch_labels = batch_labels.long().to(device)
        out = model(batch_graphs, batch_graphs.ndata["feature"].to(dtype=torch.float32))
        pred += out.argmax(dim=1).tolist()
        lab += batch_labels.tolist()

100%|██████████| 39/39 [00:01<00:00, 31.11it/s]


In [53]:
import sklearn

In [54]:
sklearn.metrics.confusion_matrix(lab, pred)

array([[122,  46,  93],
       [101,  54, 100],
       [  9,  13,  73]], dtype=int64)